# Introduction

In this tutorial, we will explain the basics of extending Data Detective to new data types, new validator methods, new validators, and new transforms. We will build up to performing anomaly detection on the MNIST dataset using PCA anomaly scoring.

Prerequisites include all of the information in the Data Detective Basics tutorial.

Let's get started!

In [ ]:
%pip install --upgrade pyod

In [ ]:
import numpy as np
import os
import pyod
import sys
import torch
import torchvision.transforms as transforms

src_path = os.path.abspath(os.path.join(os.pardir, '.'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

from typing import Dict, Set, Type

from src.aggregation.rankings import ResultAggregator, RankingAggregationMethod
from src.data_detective_engine import DataDetectiveEngine
from src.enums.enums import DataType, ValidatorMethodParameter
from src.validators.data_validator import DataValidator


# Dataset Construction

In [ ]:
from src.datasets.tutorial_dataset import TutorialDataset


dataset = TutorialDataset(
    root='./data/MNIST',
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

# Creating a Validator Method

## The Structure of Validator Methods

Each validator method is a static class that has 4 static, functional methods:
1. The `datatype` method, which returns a set of datatypes that are supported by the validator method. 
<!---The set is considered an "OR" relation (that is, if any of of the datatypes in the set are present in the dataset, the validator method will be applied). --->
2. The `param_keys` method, which returns a set containinng the data splits that the method applies to. 
3. The `validate` method, which returns some type of actionable result. 
4. The `get_method_kwargs` method, which takes in the data object and validator kwargs and sets up the calls to `validate`. 

Let's go through a simple validator method and examine all of these components. The example validator method that we will be examining determines the principle components of a multidimensional column over a dataset and uses reconstruction error over the fitted principle components to provide an anomaly score. 

## Example: PCA Anomaly Validator Method

The first step in writing a new validator method is creating a good test for the validator method using synthetic data. Tests are a crucial part of the data detective validator method construction process for three reasons: 
1. They are helpful early on in the design process for considering and enforcing sensible top-down interface decisions.
2. They are a useful piece of documentation to both yourself as you write the method and to an end user in understanding how to use your method.
3. They verify correctness of your implementation. 

For our example, we will be constructing a 10-dimensional synthetic normal dataset with 99% of samples drawn from N(μ=0, σ=1) and 1% of samples drawn from N(μ=10, σ=1). In order to examine correctness, we will look at the AUCRoC scores between the true anomaly labels and the incorrect anomaly labels. The test that we will be using is shown below. 

### 1. `datatype()` method

We would like our PCA method to take in only multidimensional data, so let's specify that in the `datatype()` method. We specify this by returning a set of `DataType` objects. If we had wanted to start extending support to new datatypes, we would at this point extend the `DataType` enumeration and specify the new data type in the `datatype` method. 

In [ ]:
def datatype() -> Set[DataType]:
    """
    @return: the datatype the validators method operates on.
    """
    return { DataType.MULTIDIMENSIONAL }

### 2. `param_keys()` method

PCA Anomaly validation is an unsupervised method, so it needs to take in the entire dataset to fit/evaluate the model on.

In [ ]:
def param_keys() -> Set[ValidatorMethodParameter]:
    """
    Lists the data splits that the validators operates on.
    
    @return: a set of data splits for the .validate() method.
    """
    return { ValidatorMethodParameter.ENTIRE_SET }

### 3. `validate()` method

Our `validate()` method will map us from some representation of the data to a single result. For the PCA `validate()` method, let's choose to take in the entire n x d data matrix for a given column of data and an option indicating the number of components to keep for computation of outlier scores. In the method body, we will fit an existing PCA anomaly detection method from `pyod` and use that model to give us a set of anomaly scores based on reconstruction loss.


In [ ]:
from pyod.models.pca import *

def validate(
    data_matrix: Type[np.array] = None, # n x d data matrix for a givenn column
    n_components=None,
) -> object:
    """
    Runs PCA anomaly detection.

    @return: a list of n scores, one per sample. 
    """
    model = pyod.models.pca.PCA(n_components=n_components)
    model.fit(data_matrix)

    anomaly_scores = model.decision_function(data_matrix)

    # mapping output results to sample ids.
    return dict(enumerate(anomaly_scores))

### 4. `get_method_kwargs()` method

In the `get_method_kwargs()` method, we will be taking the set of options passed in the validation schema as well as the data object and setting up the calls to the `validate()` method. This method should return a dictionary where each value contains the kwargs for a `validate()` call and each key reflects where the `validate()` call will store its results in the final method results dictionary. 

Every `get_method_kwargs()` method accepts two things: the validation schema and the data object. For our PCA anomaly example, we will want to perform one call for each entry in the data object, giving us a score for each column of each sample. 

It is helpful to know for this particular method setup that every dataset contains a `get_matrix` attribute that can be overridden and further optimized to retrieve a matrix representation of the Data Detective Dataset without iteration or dataloading. This is especially useful for working with parquet or CSV data when you need to optimize your setup for performance over flexibility.

<!--Every `get_method_kwargs()` method accepts two things: the validation schema and the (filtered) data object. The data object is preliminarily filtered in two ways: 
1. The `include` option in the validation schema accepts a list of regular expressions under each 
2. The `datatype()` method results in the data object being filtered to only include columns in that data object.-->



Let's write our `get_method_kwargs()` function, which needs to retrieve our `data_matrix` and `n_components` parameters. 

In [ ]:
def get_method_kwargs(data_object: Dict[str, torch.utils.data.Dataset], validator_kwargs: Dict = None) -> Dict:
    """
    Gets the arguments for each run of the validator_method, and what to store the results under.

    @param data_object: the datasets object containing the datasets (train, test, entire, etc.)
    @param validator_kwargs: the kwargs from the validation schema.
    @return: a dict mapping from the key the result from calling .validate() on the kwargs values.
    """
    entire_dataset: torch.utils.data.Dataset = data_object["entire_set"]
    matrix_dict = entire_dataset.get_matrix(column_wise=True)
        
    kwargs_dict = {
        f"{column}_results": {
            "data_matrix": column_data,
            "n_components": validator_kwargs.get("n_components")
                            # ^will default to None if n_components is not provided
        } for column, column_data in matrix_dict.items()
    }

    return kwargs_dict

Great! Let's wrap all of the methods we have written in a single class.

In [ ]:
import numpy as np

from typing import Set, Dict, Type, Union

from src.datasets.data_detective_dataset import DataDetectiveDataset
from src.enums.enums import DataType, ValidatorMethodParameter
from src.validator_methods.data_validator_method import DataValidatorMethod

class MyPCAAnomalyValidatorMethod(DataValidatorMethod):
    """
    A method for determining anomalies on multidimensional data. Operates on continuous datasets.
    """
    @staticmethod
    def name() -> str: 
        return "my_pca_validator_method"

    @staticmethod
    def datatype() -> Set[DataType]:
        return datatype()


    @staticmethod
    def param_keys() -> Set[ValidatorMethodParameter]:
        """
        @return: a set of data splits for the data object to include.
        """
        return param_keys()

    @staticmethod
    def get_method_kwargs(data_object: Dict[str, Union[DataDetectiveDataset, Dict[str, DataDetectiveDataset]]], validator_kwargs: Dict = None) -> Dict:
        """
        Gets the arguments for each run of the validator_method, and what to store the results under.

        @param data_object: the datasets object containing the datasets (train, test, entire, etc.)
        @param validator_kwargs: the kwargs from the validation schema.
        @return: a dict mapping from the key the result from calling .validate() on the kwargs values.
        """
        return get_method_kwargs(data_object, validator_kwargs)

    @staticmethod
    def validate(
        data_matrix: Type[np.array] = None, # n x d data matrix for a givenn column
        n_components=None,
    ) -> object:
        """
        Runs PCA anomaly detection.

        @return: a list of n scores, one per sample. 
        """
        return validate(data_matrix, n_components)

# Creating a Validator

Validators are simply sets of validator methods. Creating a new one is relatively straightforward. They consist of the set of validator methods that they include. Let's create a non-default validator for our PCA anomaly method.

In [ ]:
class MyUnsupervisedAnomalyDataValidator(DataValidator):
    """
    A dataset has many features/columns, and each column has many ValidatorMethods that apply to it, depending on the
    datatype. A DataValidator is a collection of ValidatorMethods for a unique purpose.
    """
    @staticmethod
    def name() -> str: 
        return "my_unsupervised_anomaly_data_validator"

    @staticmethod
    def validator_methods() -> Set[Type[DataValidatorMethod]]:
        return {
            MyPCAAnomalyValidatorMethod
        }

# Creating a Transform

There are two steps to creating a new transform: 

1. Creating a new Transform class and overriding appropriate methods.
2. Registering the new transform in the transform library. 

Let's look at an example of a simple transform that maps images to their histograms.

In [ ]:
from src.transforms.embedding_transformer import Transform


class MyHistogramTransform(Transform): 
    def __init__(self, in_place: bool = False, cache_values: bool = True):        
        super().__init__(
            new_column_datatype=DataType.MULTIDIMENSIONAL,
            in_place=in_place, 
            cache_values=cache_values
        )
    
    def initialize_transform(self, transform_kwargs):
        super().initialize_transform(transform_kwargs=transform_kwargs)

        if "data_object" in transform_kwargs.keys():
            transform_kwargs.pop("data_object")
        if "column" in transform_kwargs.keys():
            transform_kwargs.pop("column")

        self.num_bins = transform_kwargs.get("bins", 10)

    def transform(self, x):
        x_norm = (x - x.min()) / (x.max() - x.min())
        return torch.FloatTensor(np.histogram(x_norm, bins=self.num_bins)[0])

    def new_column_name(self, original_name): 
        return f"histogram_{original_name}"

There are a few patterns worth noting in the above implementation. The `get_{transform}` higher order function always takes in kwargs that are passed through from the `options` parameter of the input transforms. 

The most important is the use of an inner helper function (in this case, `full_impl`) that is returned. Returning an inner function allows for one-time initialization of the backbone and of the parsing of options in kwargs. 

Now that we have our new validator and transform, let's register them with the Data Detective Engine so we can use them in our investigation and try them out!

In [ ]:

data_detective_engine = DataDetectiveEngine()

data_detective_engine.register_validator(MyUnsupervisedAnomalyDataValidator)
data_detective_engine.register_transform(MyHistogramTransform, "my_histogram")

validation_schema = {
    "validators": {
        "my_unsupervised_anomaly_data_validator": {},
    },
    "transforms": {
        "IMAGE": [{
            "name": "my_histogram",
            "in_place": "False",
            "options": {},
        }],
    }
}

data_object = {
    "entire_set": dataset
}

# Running the Data Detective Engine

Now that the full validation schema and data object are prepared, we are ready to run the Data Detective Engine.

In [ ]:
results = data_detective_engine.validate_from_schema(validation_schema, data_object)

In [ ]:
results

Great! Let's start to look at and analyze the results we've collected.

# Interpreting Results using the Built-In Rank Aggregator

In [ ]:
aggregator = ResultAggregator(results_object=results)
input_df = aggregator.aggregate_results_multimodally("my_unsupervised_anomaly_data_validator", [RankingAggregationMethod.LOWEST_RANK, RankingAggregationMethod.HIGHEST_RANK, RankingAggregationMethod.ROUND_ROBIN])
input_df

In [ ]:
dataset.show_datapoint(0)